<a href="https://colab.research.google.com/github/juglar-diaz/SentimentAnalysis/blob/master/SentiPred_SocMedia_RNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Intro

In [0]:
# Load the Drive helper and mount
from google.colab import drive

# This will prompt for authorization.
drive.mount('/content/drive', force_remount=True)

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
!pip install stop_words

  Created wheel for stop-words: filename=stop_words-2018.7.23-cp36-none-any.whl size=32916 sha256=365ec90e98633dec51eadb43f8f548bf51823b99f95937e305b025d165c454d8
  Stored in directory: /root/.cache/pip/wheels/75/37/6a/2b295e03bd07290f0da95c3adb9a74ba95fbc333aa8b0c7c78
Successfully built stop-words


In [0]:
#!pip install -q tensorflow-text

In [0]:
!pip install -q tensorflow-gpu==2.0.0-beta1
#!pip install -q tf-nightly-gpu-2.0-preview


     |████████████████████████████████| 348.9MB 50kB/s 
     |████████████████████████████████| 501kB 44.3MB/s 
     |████████████████████████████████| 3.1MB 40.2MB/s 


In [0]:
import tensorflow as tf
import tensorflow_datasets as tfds


In [0]:
from nltk.tokenize import word_tokenize
from sklearn.model_selection import KFold, train_test_split

from sklearn import preprocessing
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn import metrics

import warnings; warnings.simplefilter('ignore')
from collections import Counter
from stop_words import get_stop_words


In [0]:
import stop_words

In [0]:
import collections


import unicodedata
import re
import numpy as np
import os
import io
import itertools


import random
import pickle
import os
sep = os.sep
import os.path

import pandas as pd
import bisect
import time
import scipy.stats as stats

import datetime

import glob
from sklearn import metrics


In [0]:
import matplotlib.pyplot as plt


def plot_graphs(history, string):
  plt.plot(history.history[string])
  plt.plot(history.history['val_'+string])
  plt.xlabel("Epochs")
  plt.ylabel(string)
  plt.legend([string, 'val_'+string])
  plt.show()

#Download Data

In [0]:
path_data = 'drive/My Drive/Colab Notebooks/RemoteML/Data/'

In [0]:
train = pd.read_json(path_data+"dutch1.json")
#train.head()


In [0]:
all_files = glob.glob(path_data + "/*.json")

li = []

for filename in all_files:
    df = pd.read_json(filename)
    li.append(df)

df = pd.concat(li, axis=0, ignore_index=True)

In [0]:
print(set(df["sentiment"]))

{'', 'negative', nan, 'neutral', 'not sure', 'positive'}


In [0]:
df1 = df[~((df["sentiment"]== ""))]

In [0]:
df1.dropna(subset=['sentiment'],inplace=True)

In [0]:
print(set(df1["sentiment"]))


{'negative', 'not sure', 'positive', 'neutral'}


In [0]:
#df1.info()

In [0]:
df1.reset_index(inplace=True)

In [0]:
X = list(df1["content"])
le = preprocessing.LabelEncoder()
y = le.fit_transform(list(df1["sentiment"]))

Xtrain, Xtest, ytrain, ytest = train_test_split(X, y, test_size=0.2, stratify=y)
Xtrain, Xval, ytrain, yval = train_test_split(Xtrain, ytrain, test_size=0.2, stratify=ytrain)

In [0]:
Xtrain[:5]

["Oostvogel: 'Dit is een materiaal dat als goede vervanger kan dienen voor aluminium en carbon, maar voor 90 procent uit biologische grondstoffen is vervaardigd. Het voornaamste bestandsdeel van het bio-composiet is vlas, een plant met stugge vezels. Met Lina willen we laten zien dat auto’s ook op duurzame wijze lichter gemaakt kunnen worden.",
 'De ene aardappel is de andere niet maar allemaal zijn ze familie van de aubergine en tomaat. Daarom ook gevoelig voor dezelfde ziekten',
 'Maar nu zag de boerin gisteren een gaaf filmpje op feestboek. Daar gebruikt een man molshopen om in te zaaien.Dus als je de tuin of je gazon wat meer (wilde) bloemen wilt geven.....Zaai dan in een molshoop. Ook leuk om buiten je eigen tuin te doen.Hoe meer bloemen voor de bijtjes, hommels en vlinders hoe beter! De boerin strooide in deze molshoop in de boomgaard Akelei-zaden.',
 'Hi Semmy welke video heb je bekeken hoe groot de watermeloenen plant is?\ufeff | Title: RE: Ah moestuintje seizoen 2 groei update

In [0]:
tokenizer = tfds.features.text.SubwordTextEncoder.build_from_corpus(
    Xtrain, target_vocab_size=5000)


In [0]:
sample_string = 'Veel beter is het dikke lookbollen uit Arleux te kopen'

tokenized_string = tokenizer.encode(sample_string)
print ('Tokenized string is {}'.format(tokenized_string))

original_string = tokenizer.decode(tokenized_string)
print ('The original string: {}'.format(original_string))

assert original_string == sample_string

Tokenized string is [2086, 945, 10, 7, 4490, 4833, 671, 3042, 388, 231, 42, 738, 1599, 1888, 12, 673]
The original string: Veel beter is het dikke lookbollen uit Arleux te kopen


In [0]:
Xtraintoke = [tokenizer.encode(s) for s in Xtrain]
train = [(text,ytrain[i]) for (i,text) in enumerate(Xtraintoke)]
train_dataset = tf.data.Dataset.from_generator(lambda: train, (tf.int32, tf.int32), (tf.TensorShape([None]), tf.TensorShape([])))


Instructions for updating:
tf.py_func is deprecated in TF V2. Instead, there are two
    options available in V2.
    - tf.py_function takes a python function which manipulates tf eager
    tensors instead of numpy arrays. It's easy to convert a tf eager tensor to
    an ndarray (just call tensor.numpy()) but having access to eager tensors
    means `tf.py_function`s can use accelerators such as GPUs as well as
    being differentiable using a gradient tape.
    - tf.numpy_function maintains the semantics of the deprecated tf.py_func
    (it is not differentiable, and manipulates numpy arrays). It drops the
    stateful argument making all functions stateful.
    


Instructions for updating:
tf.py_func is deprecated in TF V2. Instead, there are two
    options available in V2.
    - tf.py_function takes a python function which manipulates tf eager
    tensors instead of numpy arrays. It's easy to convert a tf eager tensor to
    an ndarray (just call tensor.numpy()) but having access to eager tensors
    means `tf.py_function`s can use accelerators such as GPUs as well as
    being differentiable using a gradient tape.
    - tf.numpy_function maintains the semantics of the deprecated tf.py_func
    (it is not differentiable, and manipulates numpy arrays). It drops the
    stateful argument making all functions stateful.
    


In [0]:
Xvaltoke = [tokenizer.encode(s) for s in Xval]
val = [(text,yval[i]) for (i,text) in enumerate(Xvaltoke)]
val_dataset = tf.data.Dataset.from_generator(lambda: val, (tf.int32, tf.int32), (tf.TensorShape([None]), tf.TensorShape([])))


In [0]:
Xtesttoke = [tokenizer.encode(s) for s in Xtest]
test = [(text,ytest[i]) for (i,text) in enumerate(Xtesttoke)]
test_dataset = tf.data.Dataset.from_generator(lambda: test, (tf.int32, tf.int32), (tf.TensorShape([None]), tf.TensorShape([])))


In [0]:
iterator = iter(train_dataset)
#print(next(iterator)[0].numpy())
print(next(iterator))


(<tf.Tensor: id=81, shape=(89,), dtype=int32, numpy=
array([1279, 3334,  898,  171,   10,    8, 2994,   28,   43,  560, 1712,
       4237, 4833,   62, 2048,    2,   13,  243,  411, 1461, 1305,    2,
       4529, 2342,    5,   47,   13,  575, 4833,  522,   42,  491, 2637,
       1727, 3098,   10, 1712, 2077, 4358,    3,   39,  107, 4246,  317,
       4556, 1569, 2675,    4,    7,  569, 4846, 1830,  664, 1247, 4833,
         10, 4001, 4916,    5,    8,   48,   20, 1125, 2643, 4833, 3339,
       4916,    3,  236,  538,  205,  848,   63,  215,  326,   28, 1097,
        181,   22,   37,   15, 3686, 3965, 3010,   77,  626,   72,  185,
       4847], dtype=int32)>, <tf.Tensor: id=82, shape=(), dtype=int32, numpy=1>)


In [0]:
BUFFER_SIZE = 5000
BATCH_SIZE = 32

In [0]:
# cache the dataset to memory to get a speedup while reading from it.
train_dataset = train_dataset.cache()
train_dataset = train_dataset.shuffle(BUFFER_SIZE).padded_batch(BATCH_SIZE, padded_shapes=([None],[]))
val_dataset = val_dataset.shuffle(BUFFER_SIZE).padded_batch(BATCH_SIZE, padded_shapes=([None],[]))
test_dataset = test_dataset.shuffle(BUFFER_SIZE).padded_batch(BATCH_SIZE, padded_shapes=([None],[]))


In [0]:
iterator = iter(train_dataset)
#print(next(iterator)[0].numpy())
print(next(iterator))

(<tf.Tensor: id=140, shape=(32, 123), dtype=int32, numpy=
array([[  55,   31,  461, ...,    0,    0,    0],
       [3853,  317, 2168, ...,    0,    0,    0],
       [1669,   89,  408, ...,    0,    0,    0],
       ...,
       [1375, 4833, 1177, ...,    0,    0,    0],
       [  55,   31, 1171, ...,    0,    0,    0],
       [ 838,   88,  708, ...,    0,    0,    0]], dtype=int32)>, <tf.Tensor: id=141, shape=(32,), dtype=int32, numpy=
array([1, 2, 3, 1, 3, 1, 1, 1, 1, 3, 1, 1, 1, 1, 3, 1, 1, 1, 1, 1, 3, 1,
       2, 1, 1, 0, 1, 1, 3, 1, 3, 0], dtype=int32)>)


## Create the model

Build a `tf.keras.Sequential` model and start with an embedding layer. An embedding layer stores one vector per word. When called, it converts the sequences of word indices to sequences of vectors. These vectors are trainable. After training (on enough data), words with similar meanings often have similar vectors.

This index-lookup is much more efficient than the equivalent operation of passing a one-hot encoded vector through a `tf.keras.layers.Dense` layer.

A recurrent neural network (RNN) processes sequence input by iterating through the elements. RNNs pass the outputs from one timestep to their input—and then to the next.

The `tf.keras.layers.Bidirectional` wrapper can also be used with an RNN layer. This propagates the input forward and backwards through the RNN layer and then concatenates the output. This helps the RNN to learn long range dependencies.

In [0]:
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(tokenizer.vocab_size, 64),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64)),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(4, activation='softmax')
])

Compile the Keras model to configure the training process:

In [0]:
model.compile(loss='sparse_categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

## Train the model

In [0]:
history = model.fit(train_dataset, epochs=30,
                    validation_data=test_dataset)

Epoch 1/30
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


67/67 [==============================] - 34s 502ms/step - loss: 0.9896 - accuracy: 0.6642 - val_loss: 0.0000e+00 - val_accuracy: 0.0000e+00
Epoch 2/30
67/67 [==============================] - 6s 85ms/step - loss: 0.8376 - accuracy: 0.6755 - val_loss: 0.9296 - val_accuracy: 0.6772
Epoch 3/30
67/67 [==============================] - 6s 83ms/step - loss: 0.5799 - accuracy: 0.7731 - val_loss: 1.1506 - val_accuracy: 0.5611
Epoch 4/30
67/67 [==============================] - 6s 84ms/step - loss: 0.3686 - accuracy: 0.8599 - val_loss: 1.4005 - val_accuracy: 0.6033
Epoch 5/30
67/67 [==============================] - 6s 85ms/step - loss: 0.1988 - accuracy: 0.9340 - val_loss: 1.5665 - val_accuracy: 0.5430
Epoch 6/30
67/67 [==============================] - 6s 84ms/step - loss: 0.1563 - accuracy: 0.9458 - val_loss: 1.6044 - val_accuracy: 0.5701
Epoch 7/30
67/67 [==============================] - 6s 85ms/step - loss: 0.1337 - accuracy: 0.9538 - val_loss: 1.6030 - val_accuracy: 0.6124
Epoch 8/30
67/

## Stack two or more LSTM layers

Keras recurrent layers have two available modes that are controlled by the `return_sequences` constructor argument:

* Return either the full sequences of successive outputs for each timestep (a 3D tensor of shape `(batch_size, timesteps, output_features)`).
* Return only the last output for each input sequence (a 2D tensor of shape (batch_size, output_features)).

In [0]:
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(tokenizer.vocab_size, 64),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(
        64, return_sequences=True)),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(32)),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(4, activation='softmax')
])

In [0]:
model.compile(loss='sparse_categorical_crossentropy',
              optimizer='adam',
              metrics=["accuracy"])

In [0]:
history = model.fit(train_dataset, epochs=10,
                    validation_data=test_dataset)

Epoch 1/10
67/67 [==============================] - 58s 865ms/step - loss: 0.9621 - accuracy: 0.6646 - val_loss: 0.0000e+00 - val_accuracy: 0.0000e+00
Epoch 2/10
67/67 [==============================] - 10s 151ms/step - loss: 0.8571 - accuracy: 0.6726 - val_loss: 0.9803 - val_accuracy: 0.6697
Epoch 3/10
67/67 [==============================] - 10s 152ms/step - loss: 0.5954 - accuracy: 0.7608 - val_loss: 1.2050 - val_accuracy: 0.5943
Epoch 4/10
67/67 [==============================] - 10s 152ms/step - loss: 0.3616 - accuracy: 0.8708 - val_loss: 1.4978 - val_accuracy: 0.4992
Epoch 5/10
67/67 [==============================] - 10s 149ms/step - loss: 0.2514 - accuracy: 0.9099 - val_loss: 1.6465 - val_accuracy: 0.6486
Epoch 6/10
67/67 [==============================] - 10s 150ms/step - loss: 0.2254 - accuracy: 0.9255 - val_loss: 1.3172 - val_accuracy: 0.6154
Epoch 7/10
67/67 [==============================] - 10s 150ms/step - loss: 0.1854 - accuracy: 0.9335 - val_loss: 1.5047 - val_accuracy